In [2]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 921.8 kB/s eta 0:00:000:00:01
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=0c7df4493b3bd63209bb8d23f49fecfc827f5766c289a11700fc5204da9161ef
  Stored in directory: /home/shreyans06/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [11]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max= 200)
wiki = WikipediaQueryRun(api_wrapper = wiki_wrapper)

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import  RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
document = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 200).split_documents(docs)
vector_db = FAISS.from_documents(document , OllamaEmbeddings())
retriever = vector_db.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ed3a044d880>)

In [9]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever , "langsmith-search" , "Search for information about LangSmith")
retriever_tool.name

'langsmith-search'

In [10]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1 , doc_content_chars_max= 200)
arxiv = ArxivQueryRun(api_wrapper = arxiv_wrapper)
arxiv.name

'arxiv'

In [13]:
tools = [wiki , arxiv , retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/shreyans06/Desktop/ML/LANGCHAIN-DEMO/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith-search', description='Search for information about LangSmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7ed3b83bb6a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ed3a0

In [21]:
import os
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOllama

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

llm = ChatOllama(model="llama2")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 5


In [22]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm , tools , prompt)


In [24]:
## Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent , tools = tools , verbose= True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [25]:
agent_executor.invoke({"input": "Tell me about LangSmith"})



> Entering new AgentExecutor chain...

LangSmith is a language model that I am currently trained on, and it's designed to assist with a wide range of tasks, including but not limited to:

1. Language Translation: LangSmith can translate text from one language to another, allowing you to communicate with people who speak different languages.
2. Text Summarization: LangSmith can summarize long pieces of text into shorter, more digestible versions, saving you time and effort.
3. Writing Assistance: LangSmith can help you generate ideas for writing, provide suggestions for improvement, and even write entire paragraphs or articles for you.
4. Conversational Dialogue: LangSmith is trained on a wide range of conversations, allowing it to engage in natural-sounding dialogue with you. Whether you're chatting about your day or asking for advice, LangSmith is here to help.
5. Knowledge Retrieval: LangSmith has access to a vast amount of knowledge that it can draw upon to answer questions, provi

{'input': 'Tell me about LangSmith',
 'output': "\nLangSmith is a language model that I am currently trained on, and it's designed to assist with a wide range of tasks, including but not limited to:\n\n1. Language Translation: LangSmith can translate text from one language to another, allowing you to communicate with people who speak different languages.\n2. Text Summarization: LangSmith can summarize long pieces of text into shorter, more digestible versions, saving you time and effort.\n3. Writing Assistance: LangSmith can help you generate ideas for writing, provide suggestions for improvement, and even write entire paragraphs or articles for you.\n4. Conversational Dialogue: LangSmith is trained on a wide range of conversations, allowing it to engage in natural-sounding dialogue with you. Whether you're chatting about your day or asking for advice, LangSmith is here to help.\n5. Knowledge Retrieval: LangSmith has access to a vast amount of knowledge that it can draw upon to answer 

In [26]:
agent_executor.invoke({"input": "What is ML and why is it a hot topic?"})



> Entering new AgentExecutor chain...

Hello! I'm glad you asked about Machine Learning (ML)! It's a rapidly growing field that's revolutionizing many industries. ML is a subset of Artificial Intelligence (AI) that involves training computer systems to learn from data, without being explicitly programmed. The goal is to enable machines to make predictions or take actions based on patterns they detect in the data.

ML has become a hot topic for several reasons:

1. Accelerating technological advancements: ML algorithms are driving innovation in areas like image and speech recognition, natural language processing, recommendation systems, and autonomous vehicles. As these technologies improve, we'll see more automation, efficiency, and personalization in various industries.
2. Increased data availability: The exponential growth of digital data has created a wealth of opportunities for ML to uncover insights and make predictions. With the rise of IoT devices, social media, and other data

{'input': 'What is ML and why is it a hot topic?',
 'output': "\nHello! I'm glad you asked about Machine Learning (ML)! It's a rapidly growing field that's revolutionizing many industries. ML is a subset of Artificial Intelligence (AI) that involves training computer systems to learn from data, without being explicitly programmed. The goal is to enable machines to make predictions or take actions based on patterns they detect in the data.\n\nML has become a hot topic for several reasons:\n\n1. Accelerating technological advancements: ML algorithms are driving innovation in areas like image and speech recognition, natural language processing, recommendation systems, and autonomous vehicles. As these technologies improve, we'll see more automation, efficiency, and personalization in various industries.\n2. Increased data availability: The exponential growth of digital data has created a wealth of opportunities for ML to uncover insights and make predictions. With the rise of IoT devices,